# Customer Churn – Data Cleaning Workflow

This notebook documents the full data cleaning workflow for the **Customer Subscription & Churn** dataset.

It assumes you have a raw file named `customer_churn_raw.xlsx` in the same folder.

Main steps:
- Load the raw dataset
- Inspect structure and data quality issues
- Standardize column names
- Clean string fields (country, plan type, etc.)
- Parse and normalize dates (signup and last active)
- Convert numeric fields from messy strings to numbers
- Handle missing values
- Remove duplicates
- Basic handling of outliers in `monthly_spend`
- Export a clean, analysis-ready file as `customer_churn_clean.xlsx`


In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 140)

print('pandas version:', pd.__version__)

## 1. Load raw data

The raw data file should be named **`customer_churn_raw.xlsx`** and be placed in the same directory as this notebook.

In [ ]:
raw_path = 'customer_churn_raw.xlsx'  # update if your file name is different
df_raw = pd.read_excel(raw_path)

df_raw.head()

In [ ]:
df_raw.info()

From the initial inspection, you may see:
- Inconsistent column naming (spaces, casing)
- `Signup Date` and `Last Active` as strings with mixed formats
- Numeric columns like `Age`, `Monthly Spend`, `Num Logins` stored as `object` due to messy formatting
- Text columns (country, plan type) with trailing spaces and inconsistent casing
- Churn status as 0/1 with messy formatting in some rows

## 2. Create working copy and standardize column names

We avoid modifying the raw DataFrame directly and work on a copy. Column names are standardized to:
- strip whitespace
- convert to lowercase
- replace spaces with underscores
- remove problematic characters

In [ ]:
df = df_raw.copy()

df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r'\s+', '_', regex=True)
      .str.replace('[()]', '', regex=True)
)

df.head()

Typical standardized column names after this step:

- `customer_id`
- `signup_date`
- `last_active`
- `country`
- `age`
- `monthly_spend`
- `num_logins`
- `churned`
- `plan_type`
- `satisfaction_score`


## 3. Clean string columns (country, plan_type, etc.)

We:
- Strip leading/trailing spaces
- Replace empty strings / placeholders (`'?', 'NA', ''`) with proper `NaN`
- Normalize casing (e.g. `"Singapore "` → `"SINGAPORE"`)

In [ ]:
# Clean all object columns first
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].replace({'': np.nan, 'na': np.nan, 'NA': np.nan, '?': np.nan})

# Normalize key categorical columns
cat_cols = ['country', 'plan_type']
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].str.upper()

df[cat_cols].head()

You can additionally map values like `"MALAYSIA"`, `"SINGAPORE"`, etc. to standardized forms,
or group rare categories as needed for modeling.

## 4. Convert dates and numeric fields

The `signup_date` and `last_active` columns may have different date formats. We convert both to datetime.

Numeric columns such as `age`, `monthly_spend`, `num_logins`, `churned`, and `satisfaction_score` may contain
messy strings or currency-looking values, so we remove non-numeric characters and convert to numeric types.

In [ ]:
date_cols = ['signup_date', 'last_active']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

numeric_cols = ['age', 'monthly_spend', 'num_logins', 'churned', 'satisfaction_score']
for col in numeric_cols:
    if col in df.columns:
        df[col] = (
            df[col]
              .astype(str)
              .str.replace(r'[^0-9\.-]', '', regex=True)
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')

df[numeric_cols].head()

In [ ]:
df.info()

## 5. Handle missing values

Strategy used here:
- Rows missing critical identifiers (e.g. `customer_id`) are dropped
- Numeric fields with missing values are filled with their median
- Date fields that are critical for analysis may be dropped or left as `NaT`, depending on the use case

In [ ]:
# Drop rows with missing critical ID
if 'customer_id' in df.columns:
    df = df.dropna(subset=['customer_id'])

# Impute numeric columns with median
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

df.isna().sum()

You may choose stricter rules for certain analysis use cases (e.g. dropping rows with missing `signup_date`).

## 6. Remove duplicates

We remove exact duplicate rows to avoid double counting customer records.

In [ ]:
before_rows = len(df)
df = df.drop_duplicates()
after_rows = len(df)
print(f'Removed {before_rows - after_rows} duplicate rows')

df.head()

## 7. Basic outlier handling for `monthly_spend`

The dataset may include extreme outliers in `monthly_spend` (e.g. values 10–15× higher than typical).
We use an IQR-based cap to prevent these from dominating the analysis while retaining the records.

In [ ]:
if 'monthly_spend' in df.columns:
    q1 = df['monthly_spend'].quantile(0.25)
    q3 = df['monthly_spend'].quantile(0.75)
    iqr = q3 - q1
    upper_cap = q3 + 1.5 * iqr
    print('monthly_spend upper cap (IQR-based):', upper_cap)

    df['monthly_spend_capped'] = df['monthly_spend'].clip(upper=upper_cap)

df[['monthly_spend', 'monthly_spend_capped']].head() if 'monthly_spend' in df.columns else df.head()

As with the retail project, you can decide whether to keep both original and capped versions depending on modeling strategy.

## 8. Final checks

We run a last round of checks to confirm types, ranges, and summary statistics.

In [ ]:
df.info()

In [ ]:
df.describe(include='all').transpose().head(20)

## 9. Export cleaned dataset

Finally, we export the cleaned dataset to an Excel file named **`customer_churn_clean.xlsx`**.
This file is suitable for churn modeling, dashboards, and retention analytics.

In [ ]:
clean_path = 'customer_churn_clean.xlsx'
df.to_excel(clean_path, index=False)
print('Cleaned dataset saved to:', clean_path)

---
### Notes for Upwork / Portfolio

- This notebook can be attached in your GitHub repository as a complete example of CRM/churn data cleaning.
- It demonstrates handling of messy dates, corrupted numeric fields, categorical standardization, missing data, duplicates, and outliers.
- You can adapt it to real client datasets by updating the column names and file paths.